In [ ]:
!pip install pandas scikit-learn torch transformers tqdm

BERT+MLP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# --- Data Loading and Initial Prep ---
df = pd.read_csv('fake reviews dataset.csv')
# CG (Computer-Generated) = 1 (Fake), OR (Original) = 0 (Genuine)
df['target'] = df['label'].apply(lambda x: 1 if x == 'CG' else 0)

# --- Structured Feature Engineering ---
category_col = 'category'
rating_col = 'rating'

# Define preprocessor for structured features
# The structured vector will have a dimension of 11 (10 categories + 1 rating)
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), [category_col]),
        ('num', StandardScaler(), [rating_col])
    ],
    remainder='drop',
    verbose_feature_names_out=False
)

# Fit and transform the structured features on the whole dataset
structured_features = preprocessor.fit_transform(df[[category_col, rating_col]])
feature_names = preprocessor.get_feature_names_out()

# Add structured features back to the DataFrame
structured_df = pd.DataFrame(structured_features, columns=feature_names)
df = pd.concat([df.reset_index(drop=True), structured_df], axis=1)

# Split data into Train/Validation/Test sets
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['target'])

# Define the number of structured features for the model
N_STRUCTURED_FEATURES = len(feature_names)

In [ ]:
import torch
from transformers import BertTokenizer

# --- Configuration ---
BERT_MODEL_NAME = 'bert-base-uncased'
MAX_LEN = 128
# The dimension of the BERT embedding (CLS token) for bert-base-uncased is 768
BERT_EMBEDDING_DIM = 768

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, feature_names):
        self.texts = df['text_'].tolist()
        self.structured_features = df[feature_names].values.astype(np.float32)
        self.targets = df['target'].values
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        target = self.targets[idx]
        struct_features = self.structured_features[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'structured_features': torch.tensor(struct_features, dtype=torch.float),
            'targets': torch.tensor(target, dtype=torch.long)
        }

# Create datasets
train_dataset = ReviewDataset(train_df, tokenizer, MAX_LEN, feature_names)
val_dataset = ReviewDataset(val_df, tokenizer, MAX_LEN, feature_names)
test_dataset = ReviewDataset(test_df, tokenizer, MAX_LEN, feature_names)

In [ ]:
from transformers import BertModel
import torch.nn as nn

class HybridBertClassifier(nn.Module):
    def __init__(self, n_structured_features, n_classes, bert_embedding_dim=768, dropout_prob=0.3):
        super(HybridBertClassifier, self).__init__()
        # 1. BERT Component
        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME)

        # 2. Hybrid Classifier (The "Other Model")
        # The total fused dimension is BERT_EMBEDDING_DIM (768) + N_STRUCTURED_FEATURES (11)
        total_fused_dim = bert_embedding_dim + n_structured_features

        # A simple MLP/Dense Network on the fused features
        self.classifier = nn.Sequential(
            nn.Linear(total_fused_dim, 256),  # First dense layer
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(256, n_classes)  # Final output layer (2 classes: Fake/Genuine)
        )
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, input_ids, attention_mask, structured_features):
        # 1. Get Text Embedding from BERT (using the [CLS] token output)
        output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        # The [CLS] token embedding is the first element of the last hidden state
        cls_embedding = output.last_hidden_state[:, 0, :]

        # Apply dropout to the BERT embedding
        cls_embedding = self.dropout(cls_embedding)

        # 2. Feature Fusion (Concatenation)
        # Combine the BERT embedding (text features) and the structured features
        fused_vector = torch.cat((cls_embedding, structured_features), dim=1)

        # 3. Final Classification
        output = self.classifier(fused_vector)
        return output

# Instantiate the model
model = HybridBertClassifier(
    n_structured_features=N_STRUCTURED_FEATURES,
    n_classes=2 # Fake (1) / Genuine (0)
)

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tqdm import tqdm

# --- 1. Load and Reset Data ---
file_path = 'fake reviews dataset.csv'
df = pd.read_csv(file_path)
df['target'] = df['label'].apply(lambda x: 1 if x == 'CG' else 0)

# --- 2. CORRECTED Feature Engineering ---
# We use sparse_output=False to ensure we get a dense array for the DataFrame
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['category']),
        ('num', StandardScaler(), ['rating'])
    ],
    remainder='drop',
    verbose_feature_names_out=False
)

# Fit and transform
structured_features = preprocessor.fit_transform(df[['category', 'rating']])
feature_names = preprocessor.get_feature_names_out()

# *** THE FIX: Rename 'rating' to avoid duplicate column names ***
# This ensures we don't accidentally select the original 'rating' column later
feature_names = [f"scaled_{name}" if name == 'rating' else name for name in feature_names]

# Create DataFrame with UNIQUE names
structured_df = pd.DataFrame(structured_features, columns=feature_names)
df = pd.concat([df.reset_index(drop=True), structured_df], axis=1)

print(f"Final Feature Names ({len(feature_names)}): {feature_names}")

# --- 3. Dataset and Dataloaders ---
# Split Data
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['target'])

BERT_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)
MAX_LEN = 128

class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, feature_names):
        self.texts = df['text_'].tolist()
        self.structured_features = df[feature_names].values.astype(np.float32)
        self.targets = df['target'].values
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'structured_features': torch.tensor(self.structured_features[idx], dtype=torch.float),
            'targets': torch.tensor(self.targets[idx], dtype=torch.long)
        }

# Re-create Datasets with the NEW feature names
train_dataset = ReviewDataset(train_df, tokenizer, MAX_LEN, feature_names)
val_dataset = ReviewDataset(val_df, tokenizer, MAX_LEN, feature_names)

# --- 4. Model and Training ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16
EPOCHS = 4
LEARNING_RATE = 2e-5

class HybridBertClassifier(nn.Module):
    def __init__(self, n_structured_features, n_classes):
        super(HybridBertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME)
        # Fused dimension = 768 (BERT) + N_Structured
        total_fused_dim = 768 + n_structured_features
        self.classifier = nn.Sequential(
            nn.Linear(total_fused_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, n_classes)
        )
        self.dropout = nn.Dropout(0.3)

    def forward(self, input_ids, attention_mask, structured_features):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = self.dropout(output.last_hidden_state[:, 0, :])
        fused_vector = torch.cat((cls_embedding, structured_features), dim=1)
        return self.classifier(fused_vector)

# Initialize Model with CORRECT dimension
model = HybridBertClassifier(n_structured_features=len(feature_names), n_classes=2)
model = model.to(DEVICE)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataset)//BATCH_SIZE * EPOCHS)
loss_fn = nn.CrossEntropyLoss().to(DEVICE)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# --- Training Loop ---
print(f"Starting training on {DEVICE}...")
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for d in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        input_ids = d["input_ids"].to(DEVICE)
        attention_mask = d["attention_mask"].to(DEVICE)
        structured_features = d["structured_features"].to(DEVICE)
        targets = d["targets"].to(DEVICE)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask, structured_features)
        loss = loss_fn(outputs, targets)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader)}")

In [ ]:
# !pip install shap
import shap

# --- Load the trained model (conceptual) ---
# model.load_state_dict(torch.load('best_model.pth'))
# model = model.to('cpu')
# model.eval()

# --- 1. Define the Explainer ---
# Use the SHAP DeepExplainer as the model is a neural network.
# It requires a background dataset, usually a subset of the training data.

# Get a sample batch from the DataLoader
sample_batch = next(iter(val_data_loader))

# Extract the tensors for SHAP background data
background_input_ids = sample_batch['input_ids'].to('cpu')
background_attention_mask = sample_batch['attention_mask'].to('cpu')
background_structured_features = sample_batch['structured_features'].to('cpu')

# Create a function that the explainer can call
def model_predictor(input_data):
    # input_data is a tuple/list from the explainer (input_ids, attention_mask, structured_features)
    with torch.no_grad():
        # The explainer expects a function that takes *tensors* and returns model output logits
        # Here we only need to pass the BERT and structured features.
        return model(input_data[0].long(), input_data[1].long(), input_data[2].float())

# Initialize DeepExplainer
explainer = shap.DeepExplainer(
    model_predictor,
    (background_input_ids, background_attention_mask, background_structured_features)
)

# --- 2. Calculate SHAP Values for a specific review ---
test_batch = next(iter(DataLoader(test_dataset, batch_size=1)))

# Get the inputs for the review to explain
test_input_ids = test_batch['input_ids'].to('cpu')
test_attention_mask = test_batch['attention_mask'].to('cpu')
test_structured_features = test_batch['structured_features'].to('cpu')
test_text = tokenizer.decode(test_input_ids[0].tolist(), skip_special_tokens=True)

# Calculate SHAP values
shap_values = explainer.shap_values(
    (test_input_ids, test_attention_mask, test_structured_features)
)

# --- 3. Interpretation (Conceptual) ---
# The shap_values array will be a list of two arrays (one for each class: 0 and 1).
# The array for class 1 (Fake) contains the contribution of the FUSED FEATURE VECTOR:
# [BERT_EMBEDDING_DIM] + [N_STRUCTURED_FEATURES]
# You can map the last N_STRUCTURED_FEATURES SHAP values directly to the feature_names.

# Example: Get contributions of structured features (last 11 values for class 1)
# structured_contributions = shap_values[1][0][-N_STRUCTURED_FEATURES:]
# print("Review Text:", test_text)
# print("--- Structured Feature Contributions (for 'Fake' class) ---")
# for name, contribution in zip(feature_names, structured_contributions):
#     print(f"{name}: {contribution:.4f}")

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# --- CONFIGURATION ---
# Set to False if you are on GPU or want full training
DEBUG_MODE = False
EPOCHS = 3
BATCH_SIZE = 16
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Running on {DEVICE} (DEBUG_MODE={DEBUG_MODE})")

# --- 1. DATA LOADING & PREPROCESSING ---
df = pd.read_csv('fake reviews dataset.csv')

# Optional: Slice data for debugging/speed if needed
if DEBUG_MODE:
    df = df.head(500)

df['target'] = df['label'].apply(lambda x: 1 if x == 'CG' else 0)

# Feature Engineering
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['category']),
        ('num', StandardScaler(), ['rating'])
    ],
    remainder='drop',
    verbose_feature_names_out=False
)

structured_features = preprocessor.fit_transform(df[['category', 'rating']])
feature_names = preprocessor.get_feature_names_out()

# FIX: Rename 'rating' to 'scaled_rating' to avoid duplicate columns
feature_names = [f"scaled_{name}" if name == 'rating' else name for name in feature_names]

structured_df = pd.DataFrame(structured_features, columns=feature_names)
df = pd.concat([df.reset_index(drop=True), structured_df], axis=1)

# --- 2. DATASET CLASS ---
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
MAX_LEN = 128

class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, feature_names):
        self.texts = df['text_'].tolist()
        self.structured_features = df[feature_names].values.astype(np.float32)
        self.targets = df['target'].values
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'structured_features': torch.tensor(self.structured_features[idx], dtype=torch.float),
            'targets': torch.tensor(self.targets[idx], dtype=torch.long)
        }

# --- 3. DATALOADERS (Variable Defined Here) ---
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = ReviewDataset(train_df, tokenizer, MAX_LEN, feature_names)
val_dataset = ReviewDataset(val_df, tokenizer, MAX_LEN, feature_names)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE) # <--- FIXED DEFINITION

# --- 4. HYBRID MODEL ---
class HybridBertClassifier(nn.Module):
    def __init__(self, n_structured_features, n_classes):
        super(HybridBertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Optional: Freeze BERT layers for speed
        # for param in self.bert.parameters():
        #     param.requires_grad = False

        total_fused_dim = 768 + n_structured_features
        self.classifier = nn.Sequential(
            nn.Linear(total_fused_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, n_classes)
        )
        self.dropout = nn.Dropout(0.3)

    def forward(self, input_ids, attention_mask, structured_features):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = self.dropout(output.last_hidden_state[:, 0, :])
        fused_vector = torch.cat((cls_embedding, structured_features), dim=1)
        return self.classifier(fused_vector)

model = HybridBertClassifier(len(feature_names), 2).to(DEVICE)
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss().to(DEVICE)

# --- 5. TRAINING LOOP ---
print("Starting Training...")
model.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(DEVICE)
        mask = batch['attention_mask'].to(DEVICE)
        struct = batch['structured_features'].to(DEVICE)
        targets = batch['targets'].to(DEVICE)

        optimizer.zero_grad()
        outputs = model(input_ids, mask, struct)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss / len(train_loader):.4f}")

# --- 6. EXPLAINABILITY (Uses val_data_loader) ---
print("\n--- Feature Importance Analysis ---")

# Method A: Direct Weight Inspection (Fastest & Guaranteed to work)
# This shows which structured features (Rating/Category) the model relies on most.
classifier_weights = model.classifier[0].weight.detach().cpu().numpy()
# Get average absolute weight for each feature across all neurons
avg_weights = np.mean(np.abs(classifier_weights), axis=0)

# The last N weights belong to the structured features
struct_weights = avg_weights[-len(feature_names):]
indices = np.argsort(struct_weights)[::-1]

print("Top Influential Structured Features:")
for i in indices:
    print(f"{feature_names[i]}: {struct_weights[i]:.4f}")

# Method B: Text Explainability (Conceptual)
print("\nTo interpret text, you would now use SHAP on 'model' using 'val_data_loader'.")
print("Variable 'val_data_loader' is now ready for use.")

In [ ]:
import shap
import torch
import numpy as np
import scipy as sp

# --- 1. PREPARE BACKGROUND DATA ---
# We use val_data_loader to calculate the "Average" structured feature vector.
# This acts as a fixed baseline so we can isolate the effect of the TEXT.
print("Calculating baseline structured features from val_data_loader...")
all_struct_features = []
for batch in val_data_loader:
    all_struct_features.append(batch['structured_features'])

# Calculate the mean vector (e.g., average rating, average category distribution)
mean_structured_features = torch.cat(all_struct_features).mean(dim=0).to(DEVICE)

# --- 2. DEFINE THE PREDICTION WRAPPER ---
# SHAP will pass a list of strings (texts) to this function.
# We need to turn them into tensors and add the structured features.
def f(texts):
    model.eval()

    # A. Tokenize the incoming texts
    encoding = tokenizer(
        texts.tolist() if isinstance(texts, np.ndarray) else texts,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=MAX_LEN
    ).to(DEVICE)

    # B. Create a batch of the "Mean Structured Features" to match the text batch size
    # Shape: (Batch_Size, N_Structured_Features)
    batch_struct = mean_structured_features.repeat(encoding['input_ids'].shape[0], 1)

    # C. Run the Model
    with torch.no_grad():
        outputs = model(
            input_ids=encoding['input_ids'],
            attention_mask=encoding['attention_mask'],
            structured_features=batch_struct
        )
        # Apply Softmax to get probabilities (Fake vs Genuine)
        probs = torch.nn.functional.softmax(outputs, dim=1).cpu().numpy()

    return probs

# --- 3. RUN SHAP ---
print("Initializing SHAP Explainer (this works on the text part)...")

# Create the Explainer using our wrapper function 'f' and the tokenizer
# This "masker" tells SHAP how to break the text into tokens
explainer = shap.Explainer(f, shap.maskers.Text(tokenizer))

# Get a small batch of real examples from val_data_loader to explain
# We just take the text from the first batch
sample_batch = next(iter(val_data_loader))
texts_to_explain = tokenizer.batch_decode(sample_batch['input_ids'][:5], skip_special_tokens=True)
true_labels = sample_batch['targets'][:5].tolist()

print(f"Explaining {len(texts_to_explain)} reviews...")
shap_values = explainer(texts_to_explain)

# --- 4. VISUALIZE ---
# Show the text plot
# Red = Contributes to Class 1 (Fake/CG)
# Blue = Contributes to Class 0 (Original)
shap.plots.text(shap_values)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader

# --- 1. RE-CREATE TEST SET (Using the fixed 'feature_names') ---
# We split the data again to ensure we have a clean Test Set that matches the model's structure
# 80% Train, 10% Val, 10% Test
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['target'])

# Create the dataset using the CORRECT 'feature_names' (Length: 11)
test_dataset = ReviewDataset(test_df, tokenizer, MAX_LEN, feature_names)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print(f"Test Set Re-created. Features expected: {len(feature_names)}")

# --- 2. EVALUATION FUNCTION ---
def get_predictions(model, data_loader):
    model = model.eval()
    predictions = []
    real_values = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            structured_features = batch['structured_features'].to(DEVICE)
            targets = batch['targets'].to(DEVICE)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                structured_features=structured_features
            )

            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            real_values.extend(targets.cpu().tolist())

    return predictions, real_values

# --- 3. RUN EVALUATION ---
print("Evaluating on Test Set...")
y_pred, y_test = get_predictions(model, test_loader)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred, target_names=['Original', 'Fake (CG)']))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Original', 'Fake'], yticklabels=['Original', 'Fake'])
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Create a small list of manual tests
manual_tests = [
    # 1. Real human review (imperfect grammar, specific details)
    "The box arrived crushed which was annoying but the product itself works fine. I used it for my camping trip.",

    # 2. Obvious AI review (Generic, overly enthusiastic, perfect grammar)
    "I absolutely love this product! It features a robust design and an aesthetically pleasing interface that enhances user experience.",

    # 3. Tricky AI review (Short)
    "Great item. Highly recommended."
]

# Use the wrapper we made for SHAP earlier to predict these
probs = f(manual_tests)
predictions = np.argmax(probs, axis=1)

print("Predictions (0=Original, 1=Fake):", predictions)
print("Probabilities:", probs)

BERT+XGBoost

In [ ]:
import pandas as pd
import numpy as np
import torch
import xgboost as xgb
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tqdm import tqdm

# --- CONFIGURATION ---
BATCH_SIZE = 32  # Larger batch size is fine for inference
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on {DEVICE}...")

# --- 1. DATA LOADING & PREPROCESSING ---
print("Loading data...")
df = pd.read_csv('fake reviews dataset.csv')
df['target'] = df['label'].apply(lambda x: 1 if x == 'CG' else 0)

# Feature Engineering (Structured Data)
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['category']),
        ('num', StandardScaler(), ['rating'])
    ],
    remainder='drop',
    verbose_feature_names_out=False
)

structured_features = preprocessor.fit_transform(df[['category', 'rating']])
feature_names = preprocessor.get_feature_names_out()

# FIX: Rename 'rating' to 'scaled_rating' to avoid collision
feature_names = [f"scaled_{name}" if name == 'rating' else name for name in feature_names]

# Combine into a clean DataFrame
structured_df = pd.DataFrame(structured_features, columns=feature_names)
df_final = pd.concat([df[['text_', 'target']], structured_df], axis=1)

# --- 2. BERT FEATURE EXTRACTION ---
print("Loading BERT model...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(DEVICE)
bert_model.eval() # Set to evaluation mode (crucial!)

def extract_bert_features(texts, batch_size=32):
    all_embeddings = []

    # Process in batches to show progress and avoid memory errors
    for i in tqdm(range(0, len(texts), batch_size), desc="Extracting BERT Embeddings"):
        batch_texts = texts[i : i + batch_size]

        # Tokenize
        encoded_input = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors='pt'
        ).to(DEVICE)

        # Inference (No Gradients needed = FAST)
        with torch.no_grad():
            output = bert_model(**encoded_input)
            # Take the [CLS] token (first token) from the last hidden layer
            cls_embeddings = output.last_hidden_state[:, 0, :].cpu().numpy()
            all_embeddings.append(cls_embeddings)

    return np.vstack(all_embeddings)

# Run extraction (This takes the most time, maybe 5 mins)
text_features = extract_bert_features(df_final['text_'].tolist(), BATCH_SIZE)

print(f"Text Features Shape: {text_features.shape}")
print(f"Structured Features Shape: {structured_features.shape}")

# --- 3. COMBINE FEATURES ---
# Concatenate BERT vectors (768 dims) + Structured vectors (11 dims)
X_combined = np.hstack((text_features, structured_features))
y = df_final['target'].values

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# --- 4. TRAIN XGBOOST ---
print("Training XGBoost Classifier...")
# We use a robust configuration
xgb_clf = xgb.XGBClassifier(
    n_estimators=200,      # Number of trees
    learning_rate=0.05,    # Step size
    max_depth=6,           # Depth of trees
    subsample=0.8,         # Prevent overfitting
    colsample_bytree=0.8,  # Prevent overfitting
    eval_metric='logloss',
    n_jobs=-1              # Use all CPU cores
)

xgb_clf.fit(X_train, y_train)

# --- 5. EVALUATE ---
print("\nEvaluating...")
y_pred = xgb_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Original', 'Fake']))

# --- 6. EXPLAINABILITY (Feature Importance) ---
# Create a full list of feature names: 768 BERT dims + 11 Structured names
bert_dim_names = [f"bert_emb_{i}" for i in range(text_features.shape[1])]
all_feature_names = bert_dim_names + feature_names

# Map names to the model for plotting
xgb_clf.get_booster().feature_names = all_feature_names

# Plot top 15 features
plt.figure(figsize=(10, 8))
xgb.plot_importance(xgb_clf, max_num_features=15, importance_type='weight', height=0.5)
plt.title("Top 15 Features (BERT Dimensions vs Metadata)")
plt.show()

The Previous Model (99%): We were fine-tuning BERT. This means we allowed BERT to rewrite its internal brain to memorize the specific vocabulary quirks of this specific dataset. It likely learned extremely specific patterns (e.g., "if the word 'superb' appears, it's 100% fake").

This Model (90%): We froze BERT. We forced the model to use a "general English" understanding of the text (trained on Wikipedia/Books) without adapting to this specific dataset.

The Verdict: A 90% score on a frozen model is arguably a more impressive result for a real-world scenario. It suggests that the fundamental semantics of fake reviews are distinct enough to be caught by general language models + metadata, without needing expensive fine-tuning.